$\textbf{Set Up}$

In [1]:
import numpy as np
import cirq

$\textbf{Bob receives the matrix $P$ along with the choice of $\theta = \frac{\pi}{8}$ from Alice.}$

In [2]:
P = np.array([[0, 1, 1, 0, 0],
              [0, 1, 0, 0, 1],
              [1, 0, 1, 1, 1],
              [1, 1, 0, 0, 1],
              [0, 0, 0, 1, 0],
              [1, 0, 0, 1, 0],
              [0, 0, 1, 1, 1],
              [1, 1, 0, 1, 1],
              [1, 0, 0, 0, 0],
              [0, 1, 1, 1, 0],
              [1, 1, 0, 1, 0],
              [0, 0, 0, 0, 1],
              [0, 0, 1, 0, 0],
              [0, 1, 0, 1, 1]])

n = P.shape[1]
theta = np.pi/8

$\textbf{Bob constructs the circuit corresponding to Alice's $X$-Program description.}$

Initialise a circuit with $n$ qubits.

In [3]:
qubits = cirq.LineQubit.range(n)
circuit = cirq.Circuit()

Perform a change of basis to the diagonal basis by applying Hardamard $H$ gates to each qubit.

In [4]:
circuit.append(cirq.H(q) for q in qubits)

For each term of the $X$-Program, construct the corresponding block recursively. 

In [5]:
def x_program_block(Xs):
    if len(Xs) == 1:
        yield cirq.rz(theta)(Xs[0])
    else:
        yield cirq.CNOT(Xs[0], Xs[1])
        yield x_program_block(Xs[1:])
        yield cirq.CNOT(Xs[0], Xs[1])
        
for row in P:
    Xs = [qubits[i] for i, b in enumerate(row) if b == 1]
    circuit.append(x_program_block(Xs), strategy=cirq.InsertStrategy.NEW_THEN_INLINE)

Change back to the computational basis by applying an $H$ gate to each qubit. Then perform a measurement. 

In [6]:
circuit.append(cirq.H(q) for q in qubits)
circuit.append(cirq.measure(*qubits, key='x'))

$\textbf{Bob runs the circuit, producing a number of samples to send to Alice for verification.}$

In [7]:
simulator = cirq.Simulator()
results = simulator.run(circuit, repetitions=1000).histogram(key='x')

def binary_vector(i, n):
    return [int(b) for b in bin(i)[2:].zfill(n)]

samples = {tuple(binary_vector(i, n)):results[i] for i in range(2**n)}

samples

{(0, 0, 0, 0, 0): 609,
 (0, 0, 0, 0, 1): 15,
 (0, 0, 0, 1, 0): 16,
 (0, 0, 0, 1, 1): 11,
 (0, 0, 1, 0, 0): 14,
 (0, 0, 1, 0, 1): 24,
 (0, 0, 1, 1, 0): 4,
 (0, 0, 1, 1, 1): 8,
 (0, 1, 0, 0, 0): 6,
 (0, 1, 0, 0, 1): 14,
 (0, 1, 0, 1, 0): 11,
 (0, 1, 0, 1, 1): 20,
 (0, 1, 1, 0, 0): 21,
 (0, 1, 1, 0, 1): 10,
 (0, 1, 1, 1, 0): 13,
 (0, 1, 1, 1, 1): 3,
 (1, 0, 0, 0, 0): 22,
 (1, 0, 0, 0, 1): 4,
 (1, 0, 0, 1, 0): 10,
 (1, 0, 0, 1, 1): 9,
 (1, 0, 1, 0, 0): 2,
 (1, 0, 1, 0, 1): 13,
 (1, 0, 1, 1, 0): 2,
 (1, 0, 1, 1, 1): 13,
 (1, 1, 0, 0, 0): 5,
 (1, 1, 0, 0, 1): 22,
 (1, 1, 0, 1, 0): 19,
 (1, 1, 0, 1, 1): 33,
 (1, 1, 1, 0, 0): 17,
 (1, 1, 1, 0, 1): 3,
 (1, 1, 1, 1, 0): 23,
 (1, 1, 1, 1, 1): 4}

$\textbf{Alice now verifies that Bob's samples have the correct bias in the direction of the secret codeword s}.$

In [8]:
s = np.array([0, 1, 0, 1, 0])

bias = {
    'orthogonal': 0,
    'non-orthogonal': 0
}

for sample, count in samples.items():
    if s.dot(sample) % 2 == 0:
        bias['orthogonal'] += count
    else:
        bias['non-orthogonal'] += count

print(bias)

{'orthogonal': 829, 'non-orthogonal': 171}


$82.9\%$ of Bob's samples are orthogonal to Alice's secert codeword. After performing an appropriate hypothesis test, Alice is satisfied that Bob has a quantum computer. 